In [ ]:
import os

# Local dir where you have downloaded and saved the artifacts
local_dir = "./"

mlflow_model_dir = os.path.join(local_dir, "OpenAI-CLIP-ViT-Large-Patch14", "mlflow_model_folder")

# Show the contents of the MLFlow model folder
os.listdir(mlflow_model_dir)

# You should see a list of files such as the following:
# ['artifacts', 'conda.yaml', 'MLmodel', 'python_env.yaml', 'python_model.pkl', 'requirements.txt']

### Load the test data into a Pandas DataFrame

Load some test images into a Pandas DataFrame in order to try some predictions with it.

In [ ]:
test_image_paths = [
    "./benchmark-images/can_small.jpg",
    "./benchmark-images/beach_large.jpg",
    "./benchmark-images/beach_huge.jpg"
]

In [ ]:
import pandas as pd
import base64


def read_image(image_path):
    with open(image_path, "rb") as f:
        return f.read()

images=[
        base64.encodebytes(read_image(image_path)).decode('utf-8')
        for image_path in test_image_paths
    ]

import numpy
captions = "person, beach, mountain"
data = [[img, ""] for img in images]
test_df = pd.DataFrame(
    data=data,
    columns=["image", "text"],
)
test_df['text'].iloc[0] = captions
test_df

In [ ]:
import sys
sys.path.append(os.path.join(mlflow_model_dir, 'code'))
from mlflow_wrapper import CLIPMLFlowModelWrapper
mlflow_model_wrapper = CLIPMLFlowModelWrapper(task_type="zero-shot-image-classification")
import pickle
with open(os.path.join(mlflow_model_dir, 'python_model.pkl'), 'wb') as f:
    pickle.dump(mlflow_model_wrapper, f)

In [ ]:
import mlflow.pyfunc
pyfunc_model = mlflow.pyfunc.load_model(mlflow_model_dir)

In [ ]:
clip_model = pyfunc_model.unwrap_python_model()
pytorch_total_params = sum(p.numel() for p in clip_model._model.parameters())
print(pytorch_total_params)